# LSTM Model Type 3 Age

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from collections import defaultdict

import keras_tuner
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf

from LSTM_Model_Type3 import split_position, LSTM_FitPredict

C:\Users\asus\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Set Random Seed for Code Reproductability

In [2]:
import random
random.seed(2020)

from numpy.random import seed
seed(2020)
tf.random.set_seed(2020)

import random as python_random
python_random.seed(2020)

### Read in Raw Data & Preprocessing

In [3]:
# Read in the raw age-sex cohort data without the remainder area
# Prepare the unique sa3 names, codes for LSTM fitting selection
raw_data = pd.read_csv('../Data/true_1000_fulldata.csv')
sa3_num = len(raw_data['SA3 Code'].unique())
sa3_names = raw_data['SA3 Name'].unique()
sa3_codes = raw_data['SA3 Code'].unique()

### Split the Dataset by Standard Introuced by the External Variable

In [4]:
# Generate Quantile Information for Birth Rate and Average Total Population 
split_groups = 3
split_quantile = np.arange(0, 1, 1/split_groups).tolist()
extra_stand = pd.read_csv("../Data/Exteral_Variable.csv")
quantile_store = extra_stand.quantile(split_quantile, axis=0)

# Generate the list for Storing the Quantile Information
quantile_birth = quantile_store["Birth"].tolist()
quantile_totpop = quantile_store["Ave_Pop"].tolist()

# Collect Areas with Target External Variable Value in between the Artifitial Quantile(s)
quantile_birth_dict = {}
quantile_totpop_dict = {}
for i in range(split_groups):
    if i != (split_groups - 1):
        quantile_birth_dict[i] = extra_stand.loc[(extra_stand['Birth'] >= quantile_birth[i]) & (extra_stand['Birth'] < quantile_birth[i+1])]["Code"].tolist()
        quantile_totpop_dict[i] = extra_stand.loc[(extra_stand['Ave_Pop'] >= quantile_totpop[i]) & (extra_stand['Ave_Pop'] < quantile_totpop[i+1])]["Code"].tolist()
    else:
        quantile_birth_dict[i] = extra_stand.loc[(extra_stand['Birth'] >= quantile_birth[i])]["Code"].tolist()
        quantile_totpop_dict[i] = extra_stand.loc[(extra_stand['Ave_Pop'] >= quantile_totpop[i])]["Code"].tolist()

In [5]:
totpop_quantile_data = {}
birth_quantile_data = {}
for i in range(split_groups):
    totpop_quantile_data[i] = raw_data[raw_data['SA3 Code'].isin(quantile_totpop_dict[i])]
    birth_quantile_data[i] = raw_data[raw_data['SA3 Code'].isin(quantile_birth_dict[i])]

### Preprocessing by Splitting Population into Sex-Group

In [6]:
# Split the data by sex
age_groups = ['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39','40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74','75-79', '80-84', '85+']
male_age_groups = ['m0-4', 'm5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39', 'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74', 'm75-79', 'm80-84', 'm85+']
female_age_groups = ['f0-4', 'f5-9', 'f10-14', 'f15-19', 'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54', 'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']
population_m_dict = defaultdict(dict)
population_f_dict = defaultdict(dict)

# # save population for each year into lists
# for sa3_code in sa3_codes:
#     population_m_dict[sa3_code] = dict()
#     population_f_dict[sa3_code] = dict()
#     for year in range(1991,2012):
#         if(raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)]['Total'].size>0):
#             population_m_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][male_age_groups].values.tolist()[0]
#             population_f_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][female_age_groups].values.tolist()[0]

### Create Dataframe for Storing Prediction Result

In [7]:
output = pd.DataFrame(index = range(sa3_num*36), columns = ['Code','Area name','Sex','Age group',2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011])
for sa3_code in range(0,sa3_num):
    for age_group in range(18):
        output.loc[sa3_code*36+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Females','Age group':age_groups[age_group]}
        output.loc[sa3_code*36+18+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Males','Age group':age_groups[age_group]}

### Start LSTM Fitting & Predicting

##### Define Variable for Splitting the Full Dataset

In [8]:
# Key Year for Model Fitting
train_start = 1991
train_end = 2001

# choose a number of time steps
n_steps = 3

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 18
n_age_groups = 18

# Number of Epoch(es)
epoch = 1000

# Obtain the split position for the training sex, validation sex, and the first input_x
train_val_bounds, test_bounds = split_position(n_steps, train_start, train_end)

##### LSTM Model Defining

In [9]:
'''Function for Defining the LSTM Model'''
def lstm_model(lstm1_units, activ, optimizer, loss_fun):
    model = Sequential()
    model.add(LSTM(lstm1_units, activation=activ, input_shape=(n_steps, n_features)))
    model.add(Dense(n_age_groups))
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=["mean_absolute_percentage_error"])
    return model

##### LSTM Model Tunning

In [10]:
# Define HYPERPARAMETERS
lstm1_min = 100
lstm1_max = 1000
lstm1_step = 100
activ_functions = ["relu"]
optimizers = ["adam","adagrad"]
loss_functions = ["mse"]

'''In-Build Function for LSTM Model Tuning'''
def build_model(hp):
    lstm1_units = hp.Int("LSTM units", min_value=lstm1_min,max_value=lstm1_max,step=lstm1_step)
    activation = hp.Choice("Activation Function", activ_functions)
    optimizer = hp.Choice("Optimizer", optimizers)
    loss_fun = hp.Choice("Loss Function", loss_functions)
    model = lstm_model(lstm1_units, activation, optimizer, loss_fun)
    return model

# Generate the tuned model with keras_tuner package
tuner = keras_tuner.RandomSearch(hypermodel=build_model, objective=keras_tuner.Objective("val_mean_absolute_percentage_error", "min"))

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


##### Generate Prediction Result

In [11]:
# Generate each sex's Prediction Result of each age-cohort for areas in the quantile and store them into csv
# save population for each year into lists
for quantile in range(split_groups):
    raw_data = totpop_quantile_data[quantile]
    for sa3_code in quantile_totpop_dict[quantile]:
        population_m_dict[sa3_code] = dict()
        population_f_dict[sa3_code] = dict()
        for year in range(1991,2012):
            if(raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)]['Total'].size>0):
                population_m_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][male_age_groups].values.tolist()[0]
                population_f_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][female_age_groups].values.tolist()[0]
    output = LSTM_FitPredict(quantile_totpop_dict[quantile], population_m_dict, n_steps, train_val_bounds, test_bounds, n_features, epoch, "Males", 2002, tuner, output)
    output = LSTM_FitPredict(quantile_totpop_dict[quantile], population_f_dict, n_steps, train_val_bounds, test_bounds, n_features, epoch, "Females", 2002, tuner, output)

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorf

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorf

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorf

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorf

In [12]:
output.to_csv('../Data/Projection/Iterative_predict_output_Step1_2_Type3.csv',index=False,header=True)